# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770842768932                   -0.52    9.0         
  2   -2.772146434549       -2.88       -1.33    1.0    236ms
  3   -2.772170172634       -4.62       -2.41    1.0    279ms
  4   -2.772170638482       -6.33       -3.11    1.0    246ms
  5   -2.772170722931       -7.07       -4.53    2.0    289ms
  6   -2.772170723005      -10.13       -4.85    1.0    310ms
  7   -2.772170723013      -11.12       -5.53    1.0    287ms
  8   -2.772170723015      -11.68       -6.64    2.0    313ms
  9   -2.772170723015   +  -14.57       -6.61    1.0    322ms
 10   -2.772170723015      -13.78       -7.90    1.0    294ms
 11   -2.772170723015      -14.57       -8.04    2.0    296ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770693851268                   -0.53    9.0         
  2   -2.7

1.773557806463927

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770671742451                   -0.53    8.0         
  2   -2.772042659647       -2.86       -1.30    1.0    231ms
  3   -2.772081909203       -4.41       -2.65    1.0    306ms
  4   -2.772083413356       -5.82       -3.90    2.0    257ms
  5   -2.772083417687       -8.36       -4.56    2.0    273ms
  6   -2.772083417808       -9.92       -5.68    1.0    266ms
  7   -2.772083417811      -11.52       -6.20    2.0    587ms
  8   -2.772083417811      -13.37       -6.99    1.0    318ms
  9   -2.772083417811      -14.21       -7.81    2.0    381ms
 10   -2.772083417811      -13.91       -8.31    1.0    798ms

Polarizability via ForwardDiff:       1.7725349833387414
Polarizability via finite difference: 1.773557806463927
